In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model, Sequential

2022-07-11 21:31:43.667145: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-11 21:31:43.667178: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
############
# Functions
############

# Our 3d parameter space is the surface of a sphere centered at (1, 1, 1), with radius 1/sqrt(2)
# x, y coordinates are the x and y variances of our 2d Gaussian
# z coordinate determines the x-coordinate of the mean, mu = (z, 0)

def mean_gen(theta, phi):
    mu = 0.5 + (np.sqrt(2)/2) * np.cos(phi + np.pi / 4)
    return mu


def varx_gen(theta, phi):
    vx = 1.5 - (np.sqrt(2)/2) * np.cos(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vx


def vary_gen(theta, phi):
    vy = 1.5 - (np.sqrt(2)/2) * np.sin(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vy


# Generate n data
def spherical_data(n, thetas, phis, rand=1234):
    
    mx1, my1 = np.zeros(n), np.zeros(n)
    vx1, vy1 = np.ones(n), np.ones(n)
    
    mx2, my2 = mean_gen(thetas, phis), np.zeros(n)
    vx2, vy2 = varx_gen(thetas, phis), vary_gen(thetas, phis)
    
    x1, y1 = np.transpose(np.array([np.random.normal(mx1, vx1, size=n), 
                                    np.random.normal(my1, vy1, size=n), thetas, phis])), np.zeros(n)
    x2, y2 = np.transpose(np.array([np.random.normal(mx2, vx2, size=n), 
                                    np.random.normal(my2, vy2, size=n), thetas, phis])), np.ones(n)
    
    x, y = np.append(x1, x2, axis=0), np.append(y1, y2, axis=0)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = rand)
    
    return x_train, x_test, y_train, y_test

In [40]:
# Single Point on Sphere

n = 40000
rand_n = 12345
theta, phi = 0, 0
thetas, phis = theta * np.ones(n), phi * np.ones(n)
x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis, rand_n)

model = tf.keras.models.load_model('3dmodels/discrete_model_mth15_mph15')

In [33]:
for l in model.layers:
    l.trainable=False

inputs_hold = tf.keras.Input(shape=(1,))
simple_model = Dense(2, use_bias = False)(inputs_hold)
model3 = Model(inputs = inputs_hold, outputs = simple_model)

inputs = tf.keras.Input(shape=(2,))
inputs2 = tf.keras.layers.concatenate([inputs, model3(tf.ones_like(inputs)[:,0:1])])
hidden_layer_1 = model(inputs2)
model2 = Model(inputs = inputs, outputs = hidden_layer_1)
model2.compile(loss='categorical_crossentropy', optimizer='Adam', )

model2.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 2)]          0                                            
__________________________________________________________________________________________________
tf.ones_like_5 (TFOpLambda)     (None, 2)            0           input_25[0][0]                   
__________________________________________________________________________________________________
tf.__operators__.getitem_9 (Sli (None, 1)            0           tf.ones_like_5[0][0]             
__________________________________________________________________________________________________
model_18 (Functional)           (None, 2)            2           tf.__operators__.getitem_9[0][0] 
___________________________________________________________________________________________

In [34]:
X_train = x_train[:, 0:2]

In [35]:
for i in range(50):
    #better to make a custom callback
    model2.fit(X_train, y_train, epochs=1,batch_size=100)
    print(i,"Fitted result: ",model2.trainable_weights[:])

640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
0 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[ 1.6515626 , -0.43100438]], dtype=float32)>]
640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
1 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[2.2689826 , 0.16530964]], dtype=float32)>]
640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
2 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[2.8776183, 0.7270991]], dtype=float32)>]
640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
3 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[3.4787338, 1.2438308]], dtype=float32)>]
640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
4 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, nump

640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
39 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[26.05199 , 15.443514]], dtype=float32)>]
640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
40 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[26.687115, 15.819198]], dtype=float32)>]
640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
41 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[27.322239, 16.181808]], dtype=float32)>]
640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
42 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, numpy=array([[27.958647, 16.540522]], dtype=float32)>]
640/640 [==============================] - 1s 1ms/step - loss: 0.0000e+00
43 Fitted result:  [<tf.Variable 'dense_10/kernel:0' shape=(1, 2) dtype=float32, numpy